In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
years = ['2014', '2015', '2016', '2017', '2018']
#years = ['2019'] #Year 2019 has data from 04 to 10
#years = ['2021'] #Year 2021 has data from 04 to 09
months = ['04', '05', '06', '07', '08', '09', '10', '11']

for yr in years:
    dataframes = {}
    df_concat = pd.DataFrame()
    for mm in months:
        df = "{}-{}".format(yr,mm)
        dataframes[df] = pd.read_csv("../data/BixiMontrealRentals{}/OD_{}.csv".format(yr, df))
        if len(dataframes)==1:
            df_concat = dataframes[df]
        else:
            df_concat = pd.concat([df_concat, dataframes[df]], sort=False)
    
    if sum([v.shape[0] for k,v in dataframes.items()]) == df_concat.shape[0]:
        df_concat.to_csv("../data/BixiMontrealRentals{}/OD_{}.csv".format(yr,yr), index=False)
        
        


In [ ]:
#Rename columns to follow a similar format(OD)
years = ['2020', '2021']
dataframes = {}
for yr in years:
    dataframes[yr] = pd.read_csv("../data/BixiMontrealRentals{}/OD_{}.csv".format(yr, yr))

old_cols = list(dataframes['2021'].columns)
new_cols = list(dataframes['2020'].columns)
dataframes['2021'].rename({k:v for k,v in list(zip(old_cols, new_cols))}, inplace=True, axis=1)
dataframes['2021'].to_csv("../data/BixiMontrealRentalsGrouped/OD_{}.csv".format(2021), index=False)
   

In [ ]:
#Rename columns to follow a similar format(Stations)
years = ['2019', '2020', '2021']
dataframes = {}
for yr in years:
    dataframes[yr] = pd.read_csv("../data/BixiMontrealRentals{}/Stations_{}.csv".format(yr, yr))

old_cols_2019 = list(dataframes['2019'].columns)
old_cols_2021 = list(dataframes['2021'].columns)
new_cols = list(dataframes['2020'].columns)
dataframes['2019'].rename({k:v for k,v in list(zip(old_cols_2019, new_cols))}, inplace=True, axis=1)
dataframes['2021'].rename({k:v for k,v in list(zip(old_cols_2021, new_cols))}, inplace=True, axis=1)

for yr in years:
    dataframes[yr].to_csv("../data/BixiMontrealRentals{}/Stations_{}.csv".format(yr, yr), index=False)
        

In [ ]:
#Add a year column on Stations to avoid inconsisency in the combined Stations dataframe
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
for yr in years:
    df = pd.read_csv("../data/BixiMontrealRentals{}/Stations_{}.csv".format(yr, yr))
    df['year']=int(yr)
    df.to_csv("../data/BixiMontrealRentals{}/Stations_{}.csv".format(yr,yr), index=False)
   

Dataframes Concatenation Successful!


In [ ]:
import pandas as pd

def openCsv(filename):
    """
    Opening the CSV files & converting to Pandas DataFrames
    """
    try:
        df = pd.read_csv("{}".format(filename), delimiter = ",", low_memory=False)
        print(" ✔ Dataframe {} loaded successfully.".format(filename))
        return df
    except:
        print(" X Loading {} failed.".format(filename))
        
        
def weatherCleaner(dataframe):
    """
    This function converts raw weather data into the right timezone and timestamp format.
    """
    df = dataframe.copy()
    
    df["Date/Time (LST)"] = df["Date/Time (LST)"].astype(str) + ":00"
    # Droping extra columns
    df.drop(columns = ["Longitude (x)", "Latitude (y)", "Station Name", "Climate ID", "Year", "Month", "Day", "Time (LST)", "Temp Flag", 
                       "Dew Point Temp Flag", "Rel Hum Flag", "Precip. Amount Flag", "Dew Point Temp (°C)", "Rel Hum (%)", "Wind Spd (km/h)", "Stn Press (kPa)", "Wind Chill"], inplace = True)
    
    # Converting to pd.datetime
    df["Date/Time (LST)"] = pd.to_datetime(df["Date/Time (LST)"])


    # reordering columns
    df = df[["Date/Time (LST)", "Temp (°C)", "Precip. Amount (mm)"]]
    
    # Fill empty cells with 0
    #df.fillna(0, inplace=True)
    #replace with mean temperature all cells with nan
    df.fillna(df.mean(), inplace=True)
    
    return df


def weatherAdd(dataframe, weather_df):
    """
    This function merges weather data with the main  O-D dataframe.
    """
    df1 = dataframe.copy()
    df2 = weather_df.copy()
    df1.sort_values(by=["start_date"], inplace = True)
    df1['start_date'] = pd.to_datetime(df1['start_date'], errors='coerce')

    df2.rename(columns={"Date/Time (LST)": "start_date"}, inplace = True)
    df2['start_date'] = pd.to_datetime(df2['start_date'])
    df2.sort_values(by=["start_date"])

    new_df = pd.merge_asof(df1, df2, on='start_date')
    #new_df = df1.merge(df2, on='start_date')

    return new_df



def generate_cleaned_weather(filepath):
    years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
    months = ['04', '05', '06', '07', '08', '09', '10', '11']
    
    for yr in years:
        dataframes = {}
        df_concat = pd.DataFrame()
        for mm in months:
            df = "{}-{}".format(yr,mm)
            dataframes[df] = openCsv(f"{filepath}/MontrealRawWeatherData/en_climate_hourly_QC_7024745_{mm}-{yr}_P1H.csv")
            if len(dataframes)==1:
                df_concat = dataframes[df]
            else:
                df_concat = pd.concat([df_concat, dataframes[df]], sort=False)
        
        if sum([v.shape[0] for k,v in dataframes.items()]) == df_concat.shape[0]:
            df_concat = weatherCleaner(df_concat)
            df_concat.to_csv(f"{filepath}/BixiMontrealRentals{yr}/weather_{yr}.csv", index=False)
    

filepath = "/content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data"

In [ ]:
generate_cleaned_weather(filepath)

 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_04-2014_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_05-2014_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_06-2014_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_07-2014_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_08-2014_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.


 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_08-2015_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_09-2015_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_10-2015_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_11-2015_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7024745_04-2016_P1H.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/MontrealRawWeatherData/en_climate_hourly_QC_7

In [ ]:
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

#from google.colab import files

for year in years:
    df = openCsv(f"{filepath}/BixiMontrealRentals{year}/OD_{year}.csv")
    w_df = openCsv(f"{filepath}/BixiMontrealRentals{year}/weather_{year}.csv")
    new_df = weatherAdd(df, w_df)
    print(new_df.head(10))
    #new_df.to_csv(f"new_OD_{year}.csv", index=False)
    #files.download("new_OD_2014.csv")
    new_df.to_csv(f"drive/MyDrive/COMP599/FinalProject/BIXI-Network-Analysis/data/BixiMontrealRentals{year}/new_OD_{year}.csv", index=False)
    #new_df.head(1000).to_csv(f"{filepath}/BixiMontrealRentals{year}/new_OD_{year}_first1000.csv", index=False)

 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/BixiMontrealRentals2020/OD_2020.csv loaded successfully.
 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/BixiMontrealRentals2020/weather_2020.csv loaded successfully.
           start_date  start_station_code  ... Temp (°C)  Precip. Amount (mm)
0 2020-04-15 06:00:04                6212  ...       0.2                  0.0
1 2020-04-15 06:05:41                6369  ...       0.2                  0.0
2 2020-04-15 06:07:27                6207  ...       0.2                  0.0
3 2020-04-15 06:09:16                6202  ...       0.2                  0.0
4 2020-04-15 06:09:38                6159  ...       0.2                  0.0
5 2020-04-15 06:11:30                7025  ...       0.2                  0.0
6 2020-04-15 06:17:08                6929  ...       0.2                  0.0
7 2020-04-15 06:17:44                6175  ...       0.2                  0.0
8 2020-04

In [ ]:
transit_stops = openCsv(f"{filepath}/Transit/stops.csv")

#metro_stations = openCsv(f"{filepath}/Transit/metro_stations.csv")


 ✔ Dataframe /content/drive/My Drive/COMP599/FinalProject/BIXI-Network-Analysis/data/Transit/stops.csv loaded successfully.


In [ ]:
print(transit_stops)
metro_stations = {}
metro_stations['stop_name'] = ['stop_lat', 'stop_lon', 'icon']
icons = ['circle', 'pin', 'golf']
import csv

with open(f"{filepath}/Transit/metro_stations.csv", 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['stop_name', 'stop_lat', 'stop_lon', 'icon'])
    for index, row in transit_stops.iterrows():
        writer.writerow([row['stop_name'], row['stop_lat'], row['stop_lon'], icons[row['location_type']]])
'''
for index, row in transit_stops.iterrows():
    if row['location_type'] == 2:
        metro_stations[row['stop_name']] = [row['stop_lat'], row['stop_lon'], 'golf']

metro_stations.to_csv(f"{filepath}/Transit/metro_stations.csv", index=False)
'''


           stop_id  stop_code  ... parent_station  wheelchair_boarding
0     STATION_M118      10118  ...            NaN                    2
1               43      10118  ...   STATION_M118                    2
2            43-01      10118  ...   STATION_M118                    2
3     STATION_M120      10120  ...            NaN                    2
4               42      10120  ...   STATION_M120                    2
...            ...        ...  ...            ...                  ...
9052         60987      60987  ...            NaN                    2
9053         61121      61121  ...            NaN                    2
9054         61253      61253  ...            NaN                    1
9055         61274      61274  ...            NaN                    2
9056         58596      58596  ...            NaN                    2

[9057 rows x 9 columns]


'\nfor index, row in transit_stops.iterrows():\n    if row[\'location_type\'] == 2:\n        metro_stations[row[\'stop_name\']] = [row[\'stop_lat\'], row[\'stop_lon\'], \'golf\']\n\nmetro_stations.to_csv(f"{filepath}/Transit/metro_stations.csv", index=False)\n'

In [ ]:
i=0
for row, col in transit_stops:
    i+=1
    print(row)
    print(col)
    if i == 10:
        exit

ValueError: ignored